# Source: UCI Machine Learning Repository
- Thème: Prédiction d'abandon scolaire et réussite académique
- Établissement: Enseignement supérieur au Portugal
- URL: https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success

In [ ]:
import os
import warnings
from datetime import datetime

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    RocCurveDisplay,
    classification_report,
    confusion_matrix,
    f1_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier


warnings.filterwarnings("ignore")

df = pd.read_csv("../data/dataset.csv", delimiter=";")

## Taille du dataset

In [ ]:
print(f"Nombre d'observations: {df.shape[0]:,}")
print(f"Nombre de variables: {df.shape[1]}")

## Liste des variables

In [ ]:
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

## Typage des données

In [ ]:
df.info()

In [ ]:
## Description du dataset
df.describe()

## Distribution de la variable cible (Target)

In [ ]:
target_counts = df["Target"].value_counts()
target_pct = df["Target"].value_counts(normalize=True) * 100

for category in target_counts.index:
    count = target_counts[category]
    pct = target_pct[category]
    print(f"- {category}: {count:,} étudiants ({pct:.1f}%)")

In [ ]:
sns.set_palette("Set2")
plt.figure(figsize=(8, 6))

# Création du pie chart
# target_counts = df['Target'].value_counts()
colors = ["#66c2a5", "#fc8d62", "#8da0cb"]  # Couleurs harmonieuses
explode = (0.05, 0.05, 0.05)  # Légère séparation des parts

plt.pie(
    target_counts.values,
    labels=target_counts.index,
    autopct="%1.1f%%",
    startangle=90,
    colors=colors,
    explode=explode,
    shadow=True,
    textprops={"fontsize": 12, "weight": "bold"},
)

plt.title("Distribution de la variable cible (Target)", fontsize=14, weight="bold", pad=20)

plt.axis("equal")  # Pour avoir un cercle parfait
plt.tight_layout()
plt.show()

## Premieres lignes

In [ ]:
df.head()

In [ ]:
# Analyse binaire: Dropout vs Non-Dropout
print("🎯 Classification Binaire: Dropout vs Non-Dropout\n")

# Créer une nouvelle variable binaire
df["Dropout_Binary"] = df["Target"].apply(lambda x: "Dropout" if x == "Dropout" else "Non-Dropout")

# Calculer les proportions
binary_counts = df["Dropout_Binary"].value_counts()
binary_pct = df["Dropout_Binary"].value_counts(normalize=True) * 100

print("Distribution:")
for category in binary_counts.index:
    count = binary_counts[category]
    pct = binary_pct[category]
    print(f"  • {category}: {count:,} étudiants ({pct:.1f}%)")

print(f"\nTotal: {binary_counts.sum():,} étudiants")

# Détail de la composition de Non-Dropout
print("\n📋 Composition de 'Non-Dropout':")
non_dropout_detail = df[df["Dropout_Binary"] == "Non-Dropout"]["Target"].value_counts()
for category in non_dropout_detail.index:
    count = non_dropout_detail[category]
    pct = (count / binary_counts["Non-Dropout"]) * 100
    print(f"  • {category}: {count:,} ({pct:.1f}% des Non-Dropout)")

In [ ]:
# Visualisation: Dropout vs Non-Dropout avec pie chart
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Graphique 1: Distribution binaire
colors_binary = ["#e74c3c", "#2ecc71"]  # Rouge pour Dropout, Vert pour Non-Dropout
explode_binary = (0.1, 0)  # Faire ressortir Dropout

axes[0].pie(
    binary_counts.values,
    labels=binary_counts.index,
    autopct="%1.1f%%",
    startangle=90,
    colors=colors_binary,
    explode=explode_binary,
    shadow=True,
    textprops={"fontsize": 12, "weight": "bold"},
)

axes[0].set_title("Classification Binaire\nDropout vs Non-Dropout", fontsize=14, weight="bold")

# Graphique 2: Distribution originale (rappel)
colors_original = ["#66c2a5", "#fc8d62", "#8da0cb"]
target_counts = df["Target"].value_counts()
explode_original = (0.05, 0.05, 0.05)

axes[1].pie(
    target_counts.values,
    labels=target_counts.index,
    autopct="%1.1f%%",
    startangle=90,
    colors=colors_original,
    explode=explode_original,
    shadow=True,
    textprops={"fontsize": 12, "weight": "bold"},
)

axes[1].set_title("Classification Multi-classe\n(Original)", fontsize=14, weight="bold")

plt.tight_layout()
plt.show()

print("\n💡 Comparaison:")
print(
    f"   Approche binaire: {binary_pct['Dropout']:.1f}% Dropout vs {binary_pct['Non-Dropout']:.1f}% Non-Dropout"
)
print(f"   Ratio de déséquilibre: 1:{binary_counts['Non-Dropout'] / binary_counts['Dropout']:.2f}")

## Phase 2 : Qualité des Données

Avant d'analyser les relations entre variables, vérifions la qualité de nos données :
1. **Valeurs manquantes** - Y a-t-il des données absentes ?
2. **Doublons** - Des lignes sont-elles dupliquées ?
3. **Outliers** - Des valeurs aberrantes existent-elles ?

## 2.1 Analyse des valeurs manquantes

In [ ]:
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({"Manquantes": missing, "Pourcentage (%)": missing_pct.round(2)})
missing_df = missing_df[missing_df["Manquantes"] > 0].sort_values("Manquantes", ascending=False)

if not len(missing_df):
    print("✅ Aucune valeur manquante dans le dataset !")
    print(f"- Toutes les {df.shape[1]} colonnes sont complètes")
    print(f"- {df.shape[0]:,} lignes sans données manquantes")
else:
    print("⚠️ Valeurs manquantes détectées:\n")
    display(missing_df)

# Résumé rapide
total_cells = df.shape[0] * df.shape[1]
total_missing = df.isnull().sum().sum()
total_ratio = total_missing / total_cells
print(
    f"Résumé: {total_missing:,} valeurs manquantes sur {total_cells:,} cellules ({total_ratio:.2%})"
)

### 2.2 Analyse des valeurs dupliquées

In [ ]:
# Doublons complets (toutes les colonnes identiques)
duplicates_full = df.duplicated().sum()
print(
    f"🔍 Lignes complètement dupliquées: {duplicates_full} ({(duplicates_full / len(df)) * 100:.2f}%)"
)

if duplicates_full > 0:
    print("Exemples de doublons:")
    display(df[df.duplicated(keep=False)].head(10))
else:
    print("- Aucun doublon complet détecté ✅")

# Vérifier aussi les doublons partiels (sans la colonne Target)
cols_without_target = [col for col in df.columns if col not in ["Target", "Dropout_Binary"]]
duplicates_partial = df.duplicated(subset=cols_without_target).sum()
print(f"\n🔍 Doublons partiels (mêmes features, Target différent): {duplicates_partial}")

if duplicates_partial:
    print("- Des étudiants avec les mêmes caractéristiques ont des résultats différents")
    print("- Cela peut indiquer de la variabilité naturelle ou des erreurs de saisie")

## Détection des outliers - Variables numériques

In [ ]:
# Palette de couleurs pour éviter les warnings
colors = {"Dropout": "#e74c3c", "Non-Dropout": "#2ecc71"}
order = ["Dropout", "Non-Dropout"]

# Sélection des variables numériques principales
numeric_cols = [
    "Age at enrollment",
    "Admission grade",
    "Previous qualification (grade)",
    "Curricular units 1st sem (grade)",
    "Curricular units 2nd sem (grade)",
]

# Création des boxplots avec seaborn (évite les warnings)
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    ax = axes[i]
    sns.boxplot(
        data=df,
        x="Dropout_Binary",
        y=col,
        hue="Dropout_Binary",
        hue_order=order,
        palette=colors,
        legend=False,
        ax=ax,
    )
    ax.set_title(col, fontsize=11, weight="bold")
    ax.set_xlabel("")
    ax.set_ylabel("Valeur")

# Supprimer le dernier subplot vide
axes[-1].axis("off")

plt.suptitle(
    "Boxplots des Variables Numériques Clés\n(Comparaison Dropout vs Non-Dropout)",
    fontsize=14,
    weight="bold",
    y=1.02,
)
plt.tight_layout()
plt.show()

print("\n💡 Interprétation des boxplots:")
print("- Les points au-delà des moustaches sont des outliers potentiels")
print("- Comparez les médianes (ligne horizontale) entre Dropout et Non-Dropout")
print("- Des différences marquées suggèrent un pouvoir prédictif de la variable")

In [ ]:
# 2.3b Quantification des outliers avec la méthode IQR
# ⚠️ UNIQUEMENT sur les variables CONTINUES (pas les catégorielles encodées)
print("📊 Quantification des Outliers (Méthode IQR)\n" + "=" * 50)


def count_outliers_iqr(df, column):
    """Compte les outliers selon la méthode IQR"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound


# Variables CONTINUES uniquement (exclure les catégorielles encodées en numérique)
continuous_cols = [
    "Age at enrollment",
    "Admission grade",
    "Previous qualification (grade)",
    "Curricular units 1st sem (credited)",
    "Curricular units 1st sem (enrolled)",
    "Curricular units 1st sem (evaluations)",
    "Curricular units 1st sem (approved)",
    "Curricular units 1st sem (grade)",
    "Curricular units 1st sem (without evaluations)",
    "Curricular units 2nd sem (credited)",
    "Curricular units 2nd sem (enrolled)",
    "Curricular units 2nd sem (evaluations)",
    "Curricular units 2nd sem (approved)",
    "Curricular units 2nd sem (grade)",
    "Curricular units 2nd sem (without evaluations)",
    "Unemployment rate",
    "Inflation rate",
    "GDP",
]

# Variables catégorielles exclues (même si encodées en numérique)
categorical_excluded = [
    "Marital status",
    "Application mode",
    "Application order",
    "Course",
    "Daytime/evening attendance",
    "Previous qualification",
    "Nacionality",
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation",
    "Displaced",
    "Educational special needs",
    "Debtor",
    "Tuition fees up to date",
    "Gender",
    "Scholarship holder",
    "International",
]

print(f"📋 Analyse de {len(continuous_cols)} variables continues")
print(f"   (Variables catégorielles exclues: {len(categorical_excluded)})\n")

outlier_summary = []
for col in continuous_cols:
    if col in df.columns:
        count, lower, upper = count_outliers_iqr(df, col)
        pct = (count / len(df)) * 100
        if count > 0:
            outlier_summary.append(
                {
                    "Variable": col,
                    "Outliers": count,
                    "Pourcentage": f"{pct:.2f}%",
                    "Borne inf": f"{lower:.2f}",
                    "Borne sup": f"{upper:.2f}",
                }
            )

outlier_df = pd.DataFrame(outlier_summary)
outlier_df = outlier_df.sort_values("Outliers", ascending=False)

if len(outlier_df) > 0:
    print("⚠️ Variables continues avec outliers détectés:\n")
    display(outlier_df)
    print(
        "\n💡 Note: Ces outliers peuvent être légitimes (ex: étudiants plus âgés, excellentes notes)"
    )
else:
    print("✅ Aucun outlier significatif détecté dans les variables continues")

## Phase 3 : Analyse Bivariée - Variables Numériques vs Dropout

**Objectif** : Identifier quelles variables numériques différencient les étudiants qui abandonnent de ceux qui réussissent.

**Questions clés** :
- Les Dropout ont-ils des notes d'admission plus faibles ?
- L'âge est-il un facteur de risque ?
- La performance au 1er semestre prédit-elle l'abandon ?

In [ ]:
# 3.1a Notes d'admission et qualification antérieure vs Dropout
print("📊 Notes d'Admission vs Dropout\n" + "=" * 50)

# Palette de couleurs pour éviter les warnings
colors = {"Dropout": "#e74c3c", "Non-Dropout": "#2ecc71"}
order = ["Dropout", "Non-Dropout"]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Admission grade
sns.violinplot(
    data=df,
    x="Dropout_Binary",
    y="Admission grade",
    hue="Dropout_Binary",
    hue_order=order,
    palette=colors,
    legend=False,
    ax=axes[0],
)
axes[0].set_title("Note d'admission par groupe", fontsize=12, weight="bold")
axes[0].set_xlabel("")
axes[0].set_ylabel("Note d'admission (0-200)")

# Previous qualification grade
sns.violinplot(
    data=df,
    x="Dropout_Binary",
    y="Previous qualification (grade)",
    hue="Dropout_Binary",
    hue_order=order,
    palette=colors,
    legend=False,
    ax=axes[1],
)
axes[1].set_title("Note de qualification antérieure par groupe", fontsize=12, weight="bold")
axes[1].set_xlabel("")
axes[1].set_ylabel("Note qualification (0-200)")

plt.tight_layout()
plt.show()

# Statistiques comparatives
print("\n📈 Statistiques comparatives:")
for col in ["Admission grade", "Previous qualification (grade)"]:
    dropout_mean = df[df["Dropout_Binary"] == "Dropout"][col].mean()
    non_dropout_mean = df[df["Dropout_Binary"] == "Non-Dropout"][col].mean()
    diff = non_dropout_mean - dropout_mean
    print(f"\n   {col}:")
    print(f"- Dropout:     {dropout_mean:.1f}")
    print(f"- Non-Dropout: {non_dropout_mean:.1f}")
    print(
        f"- Différence:  {diff:+.1f} points ({'✅ Significatif' if abs(diff) > 5 else '⚪ Faible'})"
    )

In [ ]:
# 3.1b Âge à l'inscription vs Dropout
print("📊 Âge à l'inscription vs Dropout\n" + "=" * 50)

# Palette de couleurs pour éviter les warnings
colors = {"Dropout": "#e74c3c", "Non-Dropout": "#2ecc71"}
order = ["Dropout", "Non-Dropout"]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Violin plot
sns.violinplot(
    data=df,
    x="Dropout_Binary",
    y="Age at enrollment",
    hue="Dropout_Binary",
    hue_order=order,
    palette=colors,
    legend=False,
    ax=axes[0],
)
axes[0].set_title("Distribution de l'âge par groupe", fontsize=12, weight="bold")
axes[0].set_xlabel("")
axes[0].set_ylabel("Âge à l'inscription")

# Histogramme superposé
for label, color in [("Dropout", "#e74c3c"), ("Non-Dropout", "#2ecc71")]:
    subset = df[df["Dropout_Binary"] == label]["Age at enrollment"]
    axes[1].hist(subset, bins=30, alpha=0.6, label=label, color=color, density=True)

axes[1].set_title("Distribution de l'âge (densité)", fontsize=12, weight="bold")
axes[1].set_xlabel("Âge à l'inscription")
axes[1].set_ylabel("Densité")
axes[1].legend()

plt.tight_layout()
plt.show()

# Statistiques comparatives
print("\n📈 Statistiques sur l'âge:")
dropout_age = df[df["Dropout_Binary"] == "Dropout"]["Age at enrollment"]
non_dropout_age = df[df["Dropout_Binary"] == "Non-Dropout"]["Age at enrollment"]

print("\n   Dropout:")
print(f"- Moyenne: {dropout_age.mean():.1f} ans | Médiane: {dropout_age.median():.0f} ans")
print(f"- Min: {dropout_age.min():.0f} | Max: {dropout_age.max():.0f}")

print("\n   Non-Dropout:")
print(f"- Moyenne: {non_dropout_age.mean():.1f} ans | Médiane: {non_dropout_age.median():.0f} ans")
print(f"- Min: {non_dropout_age.min():.0f} | Max: {non_dropout_age.max():.0f}")

diff = dropout_age.mean() - non_dropout_age.mean()
print(
    f"\n   💡 Les Dropout sont en moyenne {abs(diff):.1f} ans {'plus âgés' if diff > 0 else 'plus jeunes'}"
)

In [ ]:
# 3.1c Performance 1er Semestre vs Dropout
print("📊 Performance 1er Semestre vs Dropout\n" + "=" * 50)

# Palette de couleurs pour éviter les warnings
colors = {"Dropout": "#e74c3c", "Non-Dropout": "#2ecc71"}
order = ["Dropout", "Non-Dropout"]

sem1_cols = [
    "Curricular units 1st sem (approved)",
    "Curricular units 1st sem (grade)",
    "Curricular units 1st sem (enrolled)",
]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, col in enumerate(sem1_cols):
    sns.boxplot(
        data=df,
        x="Dropout_Binary",
        y=col,
        hue="Dropout_Binary",
        hue_order=order,
        palette=colors,
        legend=False,
        ax=axes[i],
    )
    # Titre simplifié
    short_name = col.replace("Curricular units 1st sem ", "").replace("(", "").replace(")", "")
    axes[i].set_title(f"1er Sem: {short_name.capitalize()}", fontsize=11, weight="bold")
    axes[i].set_xlabel("")

plt.suptitle(
    "Variables du 1er Semestre - Comparaison Dropout vs Non-Dropout",
    fontsize=13,
    weight="bold",
    y=1.02,
)
plt.tight_layout()
plt.show()

# Statistiques clés
print("\n📈 Statistiques du 1er semestre:")
key_col = "Curricular units 1st sem (approved)"
dropout_val = df[df["Dropout_Binary"] == "Dropout"][key_col].mean()
non_dropout_val = df[df["Dropout_Binary"] == "Non-Dropout"][key_col].mean()

print("\n   Unités validées au 1er semestre:")
print(f"- Dropout:     {dropout_val:.2f} unités en moyenne")
print(f"- Non-Dropout: {non_dropout_val:.2f} unités en moyenne")
print(
    f"- Ratio:       {non_dropout_val / max(dropout_val, 0.01):.1f}x plus d'unités validées pour Non-Dropout"
)

key_col2 = "Curricular units 1st sem (grade)"
dropout_grade = df[df["Dropout_Binary"] == "Dropout"][key_col2].mean()
non_dropout_grade = df[df["Dropout_Binary"] == "Non-Dropout"][key_col2].mean()
print("\n   Note moyenne au 1er semestre:")
print(f"- Dropout:     {dropout_grade:.2f}/20")
print(f"- Non-Dropout: {non_dropout_grade:.2f}/20")

In [ ]:
# 3.1d Performance 2ème Semestre vs Dropout
print("📊 Performance 2ème Semestre vs Dropout\n" + "=" * 50)

# Palette de couleurs pour éviter les warnings
colors = {"Dropout": "#e74c3c", "Non-Dropout": "#2ecc71"}
order = ["Dropout", "Non-Dropout"]

sem2_cols = [
    "Curricular units 2nd sem (approved)",
    "Curricular units 2nd sem (grade)",
    "Curricular units 2nd sem (enrolled)",
]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, col in enumerate(sem2_cols):
    sns.boxplot(
        data=df,
        x="Dropout_Binary",
        y=col,
        hue="Dropout_Binary",
        hue_order=order,
        palette=colors,
        legend=False,
        ax=axes[i],
    )
    short_name = col.replace("Curricular units 2nd sem ", "").replace("(", "").replace(")", "")
    axes[i].set_title(f"2ème Sem: {short_name.capitalize()}", fontsize=11, weight="bold")
    axes[i].set_xlabel("")

plt.suptitle(
    "Variables du 2ème Semestre - Comparaison Dropout vs Non-Dropout",
    fontsize=13,
    weight="bold",
    y=1.02,
)
plt.tight_layout()
plt.show()

# Comparaison 1er vs 2ème semestre
print("\n📈 Comparaison évolution entre semestres:")
print("\n   | Groupe       | Sem 1 (approved) | Sem 2 (approved) | Évolution |")
print("   |--------------|------------------|------------------|-----------|")

for group in ["Dropout", "Non-Dropout"]:
    sem1_mean = df[df["Dropout_Binary"] == group]["Curricular units 1st sem (approved)"].mean()
    sem2_mean = df[df["Dropout_Binary"] == group]["Curricular units 2nd sem (approved)"].mean()
    evolution = sem2_mean - sem1_mean
    print(f"   | {group:12} | {sem1_mean:16.2f} | {sem2_mean:16.2f} | {evolution:+9.2f} |")

print("\n💡 Observation: Comment évolue la performance entre les deux semestres ?")

## Phase 3.2 : Variables Catégorielles vs Dropout

**Objectif** : Identifier quelles catégories sont associées à un risque plus élevé de Dropout.

**Questions clés** :
- Certains programmes ont-ils plus d'abandons ?
- Les boursiers abandonnent-ils moins ?
- Le genre, l'état civil ou les cours du soir influencent-ils le Dropout ?

In [ ]:
# 3.2a Analyse des variables binaires vs Dropout
print("📊 Variables Binaires vs Taux de Dropout\n" + "=" * 50)

# Variables binaires à analyser (sans Daytime/evening attendance qui a un nom problématique)
binary_vars = [
    ("Gender", {0: "Femme", 1: "Homme"}),
    ("Scholarship holder", {0: "Non boursier", 1: "Boursier"}),
    ("Debtor", {0: "Non débiteur", 1: "Débiteur"}),
    ("Tuition fees up to date", {0: "Non à jour", 1: "À jour"}),
    ("Displaced", {0: "Local", 1: "Déplacé"}),
    ("International", {0: "National", 1: "International"}),
]

# Calculer le taux de dropout pour chaque variable
results = []
for var, labels in binary_vars:
    for val, label in labels.items():
        subset = df[df[var] == val]
        total = len(subset)
        dropout_count = len(subset[subset["Dropout_Binary"] == "Dropout"])
        dropout_rate = (dropout_count / total * 100) if total > 0 else 0
        results.append(
            {
                "Variable": var,
                "Catégorie": label,
                "Total": total,
                "Dropout": dropout_count,
                "Taux Dropout (%)": round(dropout_rate, 1),
            }
        )

results_df = pd.DataFrame(results)

# Afficher le tableau
print("\n📋 Taux de Dropout par variable binaire:\n")
display(results_df)

# Identifier les facteurs de risque
print("\n🔴 Facteurs augmentant le risque de Dropout:")
for var, labels in binary_vars:
    rates = results_df[results_df["Variable"] == var]["Taux Dropout (%)"].values
    if len(rates) == 2 and abs(rates[0] - rates[1]) > 5:
        higher = labels[0] if rates[0] > rates[1] else labels[1]
        diff = abs(rates[0] - rates[1])
        print(f"- {var}: '{higher}' → +{diff:.1f}% de dropout")

In [ ]:
# 3.2b Visualisation des variables binaires
print("📊 Visualisation du Taux de Dropout par Variable Binaire\n" + "=" * 50)

# Variables binaires (sans Daytime/evening attendance)
binary_vars_simple = [
    "Gender",
    "Scholarship holder",
    "Debtor",
    "Tuition fees up to date",
    "Displaced",
    "International",
]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, var in enumerate(binary_vars_simple):
    ax = axes[i]

    # Calculer les pourcentages
    cross_tab = pd.crosstab(df[var], df["Dropout_Binary"], normalize="index") * 100

    # Plot
    cross_tab.plot(kind="bar", ax=ax, color=["#2ecc71", "#e74c3c"], edgecolor="black")
    ax.set_title(f"{var}", fontsize=11, weight="bold")
    ax.set_xlabel("")
    ax.set_ylabel("Pourcentage (%)")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    ax.legend(["Non-Dropout", "Dropout"], loc="upper right")
    ax.set_ylim(0, 100)

    # Ajouter la ligne de référence (taux global)
    global_dropout_rate = (df["Dropout_Binary"] == "Dropout").mean() * 100
    ax.axhline(
        y=global_dropout_rate,
        color="red",
        linestyle="--",
        alpha=0.5,
        label="Taux global",
    )

plt.suptitle(
    "Taux de Dropout vs Non-Dropout par Variable Binaire",
    fontsize=14,
    weight="bold",
    y=1.02,
)
plt.tight_layout()
plt.show()

print(f"\n📈 Taux de dropout global: {global_dropout_rate:.1f}%")
print("   (Ligne rouge pointillée = référence)")

In [ ]:
# 3.2c Programme d'études (Course) vs Dropout
print("📊 Programme d'Études vs Taux de Dropout\n" + "=" * 50)

# Mapping des codes de programme
course_mapping = {
    33: "Biofuel Production",
    171: "Animation & Multimedia",
    8014: "Social Service (soir)",
    9003: "Agronomy",
    9070: "Communication Design",
    9085: "Veterinary Nursing",
    9119: "Informatics Engineering",
    9130: "Equinculture",
    9147: "Management",
    9238: "Social Service",
    9254: "Tourism",
    9500: "Nursing",
    9556: "Oral Hygiene",
    9670: "Advertising & Marketing",
    9773: "Journalism & Communication",
    9853: "Basic Education",
    9991: "Management (soir)",
}

# Calculer le taux de dropout par programme
course_stats = (
    df.groupby("Course")
    .agg({"Dropout_Binary": lambda x: (x == "Dropout").sum(), "Target": "count"})
    .rename(columns={"Dropout_Binary": "Dropouts", "Target": "Total"})
)

course_stats["Taux Dropout (%)"] = (course_stats["Dropouts"] / course_stats["Total"] * 100).round(1)
course_stats["Programme"] = course_stats.index.map(course_mapping)
course_stats = course_stats.sort_values("Taux Dropout (%)", ascending=False)

# Visualisation
fig, ax = plt.subplots(figsize=(12, 8))
colors = [
    "#e74c3c" if x > 40 else "#f39c12" if x > 30 else "#2ecc71"
    for x in course_stats["Taux Dropout (%)"]
]

bars = ax.barh(
    course_stats["Programme"],
    course_stats["Taux Dropout (%)"],
    color=colors,
    edgecolor="black",
)
ax.set_xlabel("Taux de Dropout (%)", fontsize=12)
ax.set_title("Taux de Dropout par Programme d'Études", fontsize=14, weight="bold")
ax.axvline(
    x=global_dropout_rate,
    color="red",
    linestyle="--",
    linewidth=2,
    label=f"Moyenne ({global_dropout_rate:.1f}%)",
)
ax.legend()

# Ajouter les valeurs sur les barres
for bar, val in zip(bars, course_stats["Taux Dropout (%)"]):
    ax.text(
        bar.get_width() + 0.5,
        bar.get_y() + bar.get_height() / 2,
        f"{val}%",
        va="center",
        fontsize=9,
    )

plt.tight_layout()
plt.show()

# Résumé
print("\n🔴 Programmes à HAUT risque (>40% dropout):")
high_risk = course_stats[course_stats["Taux Dropout (%)"] > 40]
for _, row in high_risk.iterrows():
    print(
        f"- {row['Programme']}: {row['Taux Dropout (%)']}% ({row['Dropouts']}/{row['Total']} étudiants)"
    )

print("\n🟢 Programmes à FAIBLE risque (<25% dropout):")
low_risk = course_stats[course_stats["Taux Dropout (%)"] < 25]
for _, row in low_risk.iterrows():
    print(
        f"- {row['Programme']}: {row['Taux Dropout (%)']}% ({row['Dropouts']}/{row['Total']} étudiants)"
    )

In [ ]:
# 3.2d Statut Matrimonial vs Dropout
print("📊 Statut Matrimonial vs Taux de Dropout\n" + "=" * 50)

# Mapping des codes d'état civil
marital_mapping = {
    1: "Célibataire",
    2: "Marié(e)",
    3: "Veuf/Veuve",
    4: "Divorcé(e)",
    5: "Union de fait",
    6: "Séparé(e)",
}

# Calculer le taux de dropout par statut matrimonial
marital_stats = (
    df.groupby("Marital status")
    .agg({"Dropout_Binary": lambda x: (x == "Dropout").sum(), "Target": "count"})
    .rename(columns={"Dropout_Binary": "Dropouts", "Target": "Total"})
)

marital_stats["Taux Dropout (%)"] = (
    marital_stats["Dropouts"] / marital_stats["Total"] * 100
).round(1)
marital_stats["Statut"] = marital_stats.index.map(marital_mapping)
marital_stats = marital_stats.sort_values("Taux Dropout (%)", ascending=False)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1: Taux de dropout
ax1 = axes[0]
x_positions = range(len(marital_stats))
colors = [
    "#e74c3c" if x > global_dropout_rate else "#2ecc71" for x in marital_stats["Taux Dropout (%)"]
]
bars1 = ax1.bar(x_positions, marital_stats["Taux Dropout (%)"], color=colors, edgecolor="black")
ax1.axhline(y=global_dropout_rate, color="red", linestyle="--", linewidth=2)
ax1.set_ylabel("Taux de Dropout (%)")
ax1.set_title("Taux de Dropout par Statut Matrimonial", fontsize=12, weight="bold")
ax1.set_xticks(x_positions)
ax1.set_xticklabels(marital_stats["Statut"], rotation=45, ha="right")

# Graphique 2: Effectifs
ax2 = axes[1]
bars2 = ax2.bar(x_positions, marital_stats["Total"], color="steelblue", edgecolor="black")
ax2.set_ylabel("Nombre d'étudiants")
ax2.set_title("Effectifs par Statut Matrimonial", fontsize=12, weight="bold")
ax2.set_xticks(x_positions)
ax2.set_xticklabels(marital_stats["Statut"], rotation=45, ha="right")

plt.tight_layout()
plt.show()

# Tableau récapitulatif
print("\n📋 Tableau récapitulatif:")
display(marital_stats[["Statut", "Total", "Dropouts", "Taux Dropout (%)"]].reset_index(drop=True))

print(
    f"\n💡 Note: La majorité des étudiants sont célibataires ({marital_stats[marital_stats['Statut'] == 'Célibataire']['Total'].values[0]} sur {len(df)})"
)

## Phase 4 : Analyse de Corrélation

**Objectif** : Identifier les variables les plus corrélées au Dropout et détecter les multicolinéarités.

**Questions clés** :
- Quelles variables numériques sont les plus liées au Dropout ?
- Y a-t-il des variables redondantes (fortement corrélées entre elles) ?
- Quelles variables prioriser pour le machine learning ?

In [ ]:
# =============================================================================
# 4.1 Matrice de Corrélation Complète
# =============================================================================
# Création d'une variable numérique pour Dropout (1 = Dropout, 0 = Non-Dropout)
df["Dropout_Numeric"] = (df["Dropout_Binary"] == "Dropout").astype(int)

# Sélection des variables numériques pour la corrélation
# On exclut les variables catégorielles encodées en numérique
numeric_for_corr = [
    "Dropout_Numeric",  # Notre variable cible
    "Age at enrollment",
    "Admission grade",
    "Previous qualification (grade)",
    # Performance 1er semestre
    "Curricular units 1st sem (credited)",
    "Curricular units 1st sem (enrolled)",
    "Curricular units 1st sem (evaluations)",
    "Curricular units 1st sem (approved)",
    "Curricular units 1st sem (grade)",
    "Curricular units 1st sem (without evaluations)",
    # Performance 2ème semestre
    "Curricular units 2nd sem (credited)",
    "Curricular units 2nd sem (enrolled)",
    "Curricular units 2nd sem (evaluations)",
    "Curricular units 2nd sem (approved)",
    "Curricular units 2nd sem (grade)",
    "Curricular units 2nd sem (without evaluations)",
    # Indicateurs économiques
    "Unemployment rate",
    "Inflation rate",
    "GDP",
]

# Calcul de la matrice de corrélation
corr_matrix = df[numeric_for_corr].corr()

# Visualisation avec heatmap
fig, ax = plt.subplots(figsize=(16, 14))

# Masque pour la partie triangulaire supérieure (évite les doublons)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

# Heatmap avec annotations
sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    fmt=".2f",
    cmap="RdBu_r",  # Rouge = corrélation négative, Bleu = positive
    center=0,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8, "label": "Coefficient de corrélation"},
    ax=ax,
)

ax.set_title(
    "Matrice de Corrélation - Variables Numériques vs Dropout\n(Triangle inférieur)",
    fontsize=14,
    fontweight="bold",
    pad=20,
)

plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n📊 Interprétation de la heatmap :")
print("• 🔴 Rouge foncé : Corrélation négative forte (quand l'une augmente, l'autre diminue)")
print("• 🔵 Bleu foncé : Corrélation positive forte (les deux augmentent ensemble)")
print("• ⚪ Blanc/Clair : Pas de corrélation significative")

In [ ]:
# =============================================================================
# 4.2 Top 10 Corrélations avec Dropout
# =============================================================================
# Extraction des corrélations avec notre variable cible
dropout_corr = corr_matrix["Dropout_Numeric"].drop("Dropout_Numeric")

# Tri par valeur absolue (pour voir les plus fortes, positives ou négatives)
dropout_corr_sorted = dropout_corr.reindex(dropout_corr.abs().sort_values(ascending=False).index)

# Affichage du Top 10
print("🎯 Top 10 Variables les plus corrélées au Dropout\n")
print("=" * 60)
for i, (var, corr) in enumerate(dropout_corr_sorted.head(10).items(), 1):
    direction = "⬆️ positive" if corr > 0 else "⬇️ négative"
    print(f"{i:2}. {var:<45} {corr:+.3f} ({direction})")
print("=" * 60)

# Visualisation avec barplot horizontal
fig, ax = plt.subplots(figsize=(12, 8))

colors = ["#e74c3c" if x > 0 else "#2ecc71" for x in dropout_corr_sorted.head(10)]
bars = ax.barh(range(10), dropout_corr_sorted.head(10).values, color=colors)

ax.set_yticks(range(10))
ax.set_yticklabels(dropout_corr_sorted.head(10).index)
ax.invert_yaxis()  # Top variables en haut
ax.axvline(x=0, color="black", linewidth=0.5)
ax.set_xlabel("Coefficient de Corrélation avec Dropout")
ax.set_title(
    "Top 10 Variables les Plus Corrélées au Dropout\n(Rouge = ↑ Dropout, Vert = ↓ Dropout)",
    fontsize=12,
    fontweight="bold",
)

# Annotations sur les barres
for bar, val in zip(bars, dropout_corr_sorted.head(10).values):
    ax.text(
        val + 0.02 if val > 0 else val - 0.02,
        bar.get_y() + bar.get_height() / 2,
        f"{val:+.3f}",
        va="center",
        ha="left" if val > 0 else "right",
        fontsize=9,
    )

plt.tight_layout()
plt.show()

# Interprétation
print("\n📊 Interprétation :")
print("\n🔴 Corrélations POSITIVES (facteurs de risque - augmentent le dropout) :")
for var, corr in dropout_corr_sorted.head(10).items():
    if corr > 0.1:
        print(f"- {var}: {corr:+.3f}")

print("\n🟢 Corrélations NÉGATIVES (facteurs protecteurs - diminuent le dropout) :")
for var, corr in dropout_corr_sorted.head(10).items():
    if corr < -0.1:
        print(f"- {var}: {corr:+.3f}")

In [ ]:
# =============================================================================
# 4.3 Analyse de Multicolinéarité
# =============================================================================
# Recherche des paires de variables fortement corrélées entre elles
# (potentiellement redondantes pour un modèle ML)

# Création d'un masque pour la partie triangulaire supérieure (sans la diagonale)
mask_upper = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

# Extraction des corrélations
corr_pairs = []
for i in range(len(corr_matrix)):
    for j in range(i + 1, len(corr_matrix)):
        var1 = corr_matrix.index[i]
        var2 = corr_matrix.columns[j]
        corr_val = corr_matrix.iloc[i, j]
        if var1 != "Dropout_Numeric" and var2 != "Dropout_Numeric":
            corr_pairs.append((var1, var2, corr_val))

# Tri par corrélation absolue
corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

# Affichage des paires les plus corrélées
print("🔍 Paires de Variables avec Forte Multicolinéarité (|r| > 0.7)\n")
print("=" * 80)
print(f"{'Variable 1':<40} {'Variable 2':<25} {'Corrélation':>10}")
print("-" * 80)

high_corr_count = 0
for var1, var2, corr in corr_pairs:
    if abs(corr) > 0.7:
        print(f"{var1:<40} {var2:<25} {corr:+.3f}")
        high_corr_count += 1

if high_corr_count == 0:
    print("Aucune paire avec |r| > 0.7")
print("=" * 80)

print(f"\n📊 Nombre de paires avec multicolinéarité forte : {high_corr_count}")

# Recommandations
print("\n💡 Recommandations pour le Machine Learning :")
print("\n1. Variables CLÉS (plus corrélées au Dropout) :")
for var, corr in dropout_corr_sorted.head(5).items():
    print(f"- {var}: {corr:+.3f}")

print("\n2. Variables REDONDANTES (multicolinéarité) :")
seen = set()
for var1, var2, corr in corr_pairs[:5]:
    if abs(corr) > 0.5:
        if var1 not in seen and var2 not in seen:
            print(f"- {var1} ↔ {var2} (r={corr:+.3f})")
            print("     → Considérer de ne garder qu'une des deux")
            seen.add(var2)

print("\n3. Stratégie recommandée :")
print("- Prioriser les variables de performance (notes, unités validées)")
print("- Attention : les variables du 1er et 2ème semestre sont très corrélées")
print("- L'âge est le seul facteur de risque positif majeur")

### Synthèse Phase 4 : Matrice de Corrélation

**Découvertes clés :**

| Aspect | Résultat |
|--------|----------|
| **Prédicteurs les plus forts** | Notes et unités validées du 2ème semestre (r ≈ -0.57) |
| **Facteur de risque positif** | Âge à l'inscription (+0.254) - étudiants plus âgés |
| **Indicateurs économiques** | Faible corrélation avec le dropout (< 0.05) |
| **Multicolinéarité détectée** | 11 paires avec \|r\| > 0.7 |

**Implications pour le Machine Learning :**
1. Les variables sem1 et sem2 sont fortement corrélées → risque de redondance
2. Prioriser les **grades** (notes) sur les autres métriques académiques
3. Le 2ème semestre est plus prédictif que le 1er
4. L'âge est un facteur de risque à ne pas négliger

**Prochaine étape suggérée :** Feature Engineering (créer des ratios, agrégations) pour réduire la dimensionnalité tout en conservant l'information prédictive.

## Phase 5 : Feature Engineering

**Objectif** : Créer de nouvelles variables pour améliorer la prédiction et réduire la multicolinéarité détectée en Phase 4.

**Transformations prévues :**
| Type | Description |
|------|-------------|
| **Discrétisation** | Âge → tranches (17-20, 21-25, 26-35, 36+) |
| **Regroupement** | Statut marital → Solo/Couple |
| **Regroupement** | Qualifications → Secondaire/Supérieur |
| **Regroupement** | Programmes → Domaines (Santé, Tech, Business...) |
| **Ratios** | Taux de réussite par semestre |
| **Agrégations** | Notes moyennes, tendance de performance |

In [ ]:
# =============================================================================
# 5.1 Discrétisation de l'âge
# =============================================================================
# Objectif : Transformer l'âge continu en catégories pour faciliter l'analyse
# Justification : L'âge a une corrélation positive avec le dropout (+0.254)

# Définition des tranches d'âge (labels purement descriptifs)
bins = [0, 20, 25, 35, 100]
labels = ["17-20", "21-25", "26-35", "36+"]

# Création de la nouvelle variable
df["Age_Group"] = pd.cut(df["Age at enrollment"], bins=bins, labels=labels)

# Vérification de la distribution
print("📊 Distribution des tranches d'âge:")
print("=" * 50)
age_dist = df["Age_Group"].value_counts().sort_index()
for group, count in age_dist.items():
    pct = count / len(df) * 100
    print(f"  {group}: {count:,} ({pct:.1f}%)")

# Calcul du taux de dropout par tranche d'âge (avec observed=True pour éviter le FutureWarning)
print("\n🎯 Taux de Dropout par tranche d'âge:")
print("=" * 50)
dropout_by_age = (
    df.groupby("Age_Group", observed=True)["Dropout_Binary"]
    .apply(lambda x: (x == "Dropout").mean() * 100)
    .sort_index()
)

for group, rate in dropout_by_age.items():
    indicator = "🔴" if rate > 35 else "🟡" if rate > 30 else "🟢"
    print(f"  {indicator} {group}: {rate:.1f}%")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution des tranches d'âge
colors = ["#3498db", "#9b59b6", "#e67e22", "#e74c3c"]
ax1 = axes[0]
age_dist.plot(kind="bar", ax=ax1, color=colors, edgecolor="black", alpha=0.8)
ax1.set_title("Distribution des Tranches d'Âge", fontsize=14, fontweight="bold")
ax1.set_xlabel("Tranche d'âge")
ax1.set_ylabel("Nombre d'étudiants")
ax1.tick_params(axis="x", rotation=45)

# Ajouter les pourcentages sur les barres
for i, (idx, val) in enumerate(age_dist.items()):
    ax1.text(i, val + 50, f"{val / len(df) * 100:.1f}%", ha="center", fontsize=10)

# Taux de dropout par tranche
ax2 = axes[1]
colors_dropout = [
    "#2ecc71" if r < 30 else "#f39c12" if r < 35 else "#e74c3c" for r in dropout_by_age
]
dropout_by_age.plot(kind="bar", ax=ax2, color=colors_dropout, edgecolor="black", alpha=0.8)
ax2.axhline(
    y=df["Dropout_Binary"].apply(lambda x: x == "Dropout").mean() * 100,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Moyenne globale",
)
ax2.set_title("Taux de Dropout par Tranche d'Âge", fontsize=14, fontweight="bold")
ax2.set_xlabel("Tranche d'âge")
ax2.set_ylabel("Taux de Dropout (%)")
ax2.tick_params(axis="x", rotation=45)
ax2.legend()

# Ajouter les valeurs sur les barres
for i, (idx, val) in enumerate(dropout_by_age.items()):
    ax2.text(i, val + 1, f"{val:.1f}%", ha="center", fontsize=10, fontweight="bold")

plt.tight_layout()
plt.show()

print("\n💡 Insight : Les étudiants plus âgés (26+) ont un taux de dropout plus élevé.")

In [ ]:
# =============================================================================
# 5.7 Tableau récapitulatif des nouvelles features
# =============================================================================

# Recréer les features manquantes si nécessaire (en cas de redémarrage kernel)
if "Marital_Binary" not in df.columns:
    solo = [1, 3, 4, 6]  # Célibataire, Veuf, Divorcé, Séparé
    df["Marital_Binary"] = df["Marital status"].apply(lambda x: "Solo" if x in solo else "Couple")

if "Education_Level" not in df.columns:
    secondaire = [1, 9, 10, 12, 14, 15, 19, 38]
    superieur = [2, 3, 4, 5, 6, 39, 40, 42, 43]
    df["Education_Level"] = df["Previous qualification"].apply(
        lambda x: "Secondaire" if x in secondaire else "Supérieur" if x in superieur else "Autre"
    )

if "Course_Domain" not in df.columns:
    course_domains = {
        33: "Tech",
        171: "Arts",
        8014: "Social",
        9003: "Sciences",
        9070: "Arts",
        9085: "Santé",
        9119: "Tech",
        9130: "Sciences",
        9147: "Business",
        9238: "Social",
        9254: "Business",
        9500: "Santé",
        9556: "Santé",
        9670: "Business",
        9773: "Arts",
        9853: "Education",
        9991: "Business",
    }
    df["Course_Domain"] = df["Course"].map(course_domains)

if "Success_Rate_Sem1" not in df.columns:
    df["Success_Rate_Sem1"] = df["Curricular units 1st sem (approved)"] / df[
        "Curricular units 1st sem (enrolled)"
    ].replace(0, np.nan)
    df["Success_Rate_Sem2"] = df["Curricular units 2nd sem (approved)"] / df[
        "Curricular units 2nd sem (enrolled)"
    ].replace(0, np.nan)

if "Avg_Grade" not in df.columns:
    df["Avg_Grade"] = (
        df["Curricular units 1st sem (grade)"] + df["Curricular units 2nd sem (grade)"]
    ) / 2
    df["Total_Approved"] = (
        df["Curricular units 1st sem (approved)"] + df["Curricular units 2nd sem (approved)"]
    )
    df["Performance_Trend"] = (
        df["Curricular units 2nd sem (grade)"] - df["Curricular units 1st sem (grade)"]
    )

## Récapitulatif des nouvelles features créées

In [ ]:
# Liste des nouvelles features avec leurs statistiques
new_features = [
    ("Age_Group", "Catégorielle", "4 catégories"),
    ("Marital_Binary", "Binaire", "Solo / Couple"),
    ("Education_Level", "Catégorielle", "3 niveaux"),
    ("Course_Domain", "Catégorielle", "6 domaines"),
    ("Success_Rate_Sem1", "Numérique", "Ratio [0-1]"),
    ("Success_Rate_Sem2", "Numérique", "Ratio [0-1]"),
    ("Avg_Grade", "Numérique", "Moyenne notes"),
    ("Total_Approved", "Numérique", "Somme unités"),
    ("Performance_Trend", "Numérique", "Différence grades"),
]

print(f"{'Feature':<25} {'Type':<15} {'Description':<20}")
print("-" * 60)
for name, ftype, desc in new_features:
    print(f"{name:<25} {ftype:<15} {desc:<20}")

print()
print("=" * 70)
print()

# Impact sur le dropout pour chaque nouvelle feature
print("🎯 IMPACT DES NOUVELLES FEATURES SUR LE DROPOUT")
print("=" * 70)
print()

# Features catégorielles
print("VARIABLES CATÉGORIELLES:")
print("-" * 40)

for feat in ["Age_Group", "Marital_Binary", "Education_Level", "Course_Domain"]:
    dropout_rate = df.groupby(feat, observed=True)["Dropout_Binary"].apply(
        lambda x: (x == "Dropout").mean() * 100
    )
    max_rate = dropout_rate.max()
    min_rate = dropout_rate.min()
    spread = max_rate - min_rate
    print(f"  {feat}: écart {spread:.1f}% (min: {min_rate:.1f}%, max: {max_rate:.1f}%)")

print()
print("VARIABLES NUMÉRIQUES (corrélation avec Dropout):")
print("-" * 40)

for feat in [
    "Success_Rate_Sem1",
    "Success_Rate_Sem2",
    "Avg_Grade",
    "Total_Approved",
    "Performance_Trend",
]:
    corr = df[feat].corr(df["Dropout_Numeric"])
    direction = "🔴 Risque" if corr > 0 else "🟢 Protecteur"
    print(f"  {feat}: r = {corr:+.3f} ({direction})")

print()
print("=" * 70)
print("💡 Les nouvelles features améliorent la lisibilité et réduisent la multicolinéarité")

### Synthèse Phase 5 : Feature Engineering

**9 nouvelles variables créées** pour améliorer l'analyse et préparer le Machine Learning.

#### Résultats clés :

| Type de transformation | Meilleure feature | Impact sur Dropout |
|------------------------|-------------------|-------------------|
| **Discrétisation** | Age_Group | Écart de 36.3% entre tranches |
| **Regroupement catégoriel** | Course_Domain | Écart de 34.7% entre domaines |
| **Ratio de performance** | Success_Rate_Sem2 | r = -0.705 (le plus fort !) |
| **Agrégation** | Avg_Grade | r = -0.551 |

#### Découvertes importantes :

1. **Le taux de réussite est plus prédictif que les notes brutes**
   - `Success_Rate_Sem2` (r = -0.705) surpasse même les notes individuelles
   - Cela confirme l'importance du ratio approved/enrolled

2. **Les domaines d'études révèlent des patterns clairs**
   - Tech : 54.9% de dropout (risque très élevé)
   - Santé : 20.3% de dropout (protection forte)

3. **Les regroupements simplifient sans perdre d'information**
   - Solo vs Couple capture l'essentiel du statut marital
   - Secondaire/Supérieur est suffisant pour la qualification

#### Recommandations pour le Modeling :

✅ **Variables à privilégier** :
- `Success_Rate_Sem2` (r = -0.705)
- `Age_Group` (écart 36.3%)
- `Course_Domain` (écart 34.7%)

❌ **Variables redondantes à éviter** :
- Ne pas utiliser à la fois les notes ET les ratios
- Choisir entre `Total_Approved` et `Success_Rate`

---

**L'EDA est maintenant complet !** Les données sont prêtes pour la phase de modélisation.

## Phase 6 : Préparation ML et Modélisation

**Objectif** : Préparer les données et entraîner des modèles de classification pour prédire le dropout.

### Stratégie basée sur l'EDA :
| Aspect | Décision |
|--------|----------|
| **Features clés** | Ratios de performance (`Success_Rate_Sem2` r=-0.705), agrégations |
| **Exclusions** | Variables avec multicolinéarité (grades individuels), indicateurs économiques |
| **Déséquilibre** | ~32% Dropout → utiliser `class_weight='balanced'` |
| **Métrique prioritaire** | **Recall** (ne pas manquer les étudiants à risque) |

In [ ]:
# =============================================================================
# 6.1 Préparation des Features et Target
# =============================================================================
# Sélection des features basée sur l'EDA (corrélations, multicolinéarité)

# Recréer les features engineered si nécessaire
if "Success_Rate_Sem1" not in df.columns:
    df["Success_Rate_Sem1"] = df["Curricular units 1st sem (approved)"] / df[
        "Curricular units 1st sem (enrolled)"
    ].replace(0, np.nan)
    df["Success_Rate_Sem2"] = df["Curricular units 2nd sem (approved)"] / df[
        "Curricular units 2nd sem (enrolled)"
    ].replace(0, np.nan)
    df["Avg_Grade"] = (
        df["Curricular units 1st sem (grade)"] + df["Curricular units 2nd sem (grade)"]
    ) / 2
    df["Total_Approved"] = (
        df["Curricular units 1st sem (approved)"] + df["Curricular units 2nd sem (approved)"]
    )
    df["Performance_Trend"] = (
        df["Curricular units 2nd sem (grade)"] - df["Curricular units 1st sem (grade)"]
    )

# Définir les features selon le plan
numeric_features = [
    "Success_Rate_Sem1",
    "Success_Rate_Sem2",
    "Avg_Grade",
    "Total_Approved",
    "Age at enrollment",
    "Admission grade",
    "Performance_Trend",
]

# Features catégorielles créées pendant le Feature Engineering
categorical_features = [
    "Age_Group",
    "Course_Domain",
    "Marital_Binary",
    "Education_Level",
]

# Features binaires (déjà encodées 0/1)
binary_features = [
    "Tuition fees up to date",
    "Scholarship holder",
    "Debtor",
    "Gender",
    "Displaced",
]

# Vérifier que toutes les features existent
all_features = numeric_features + categorical_features + binary_features
missing_cols = [col for col in all_features if col not in df.columns]
if missing_cols:
    print(f"⚠️ Colonnes manquantes : {missing_cols}")
else:
    print("✅ Toutes les features sont disponibles")

# Créer X (features) et y (target binaire)
X = df[all_features].copy()
y = (df["Dropout_Binary"] == "Dropout").astype(int)  # 1 = Dropout, 0 = Non-Dropout

print("\nDimensions:")
print(f"  X : {X.shape}")
print(f"  y : {y.shape}")
print("\nDistribution du target:")
print(f"  Dropout (1)     : {y.sum()} ({y.mean() * 100:.1f}%)")
print(f"  Non-Dropout (0) : {len(y) - y.sum()} ({(1 - y.mean()) * 100:.1f}%)")

In [ ]:
# =============================================================================
# 6.2 Gestion des Valeurs Manquantes
# =============================================================================
# Les ratios Success_Rate peuvent avoir des NaN (division par 0 si enrolled=0)

print("📊 Valeurs manquantes dans X:")
print("=" * 50)
missing = X.isnull().sum()
missing_pct = (X.isnull().sum() / len(X)) * 100
missing_df = pd.DataFrame({"Manquantes": missing, "Pourcentage": missing_pct})
missing_df = missing_df[missing_df["Manquantes"] > 0].sort_values("Manquantes", ascending=False)

if len(missing_df) > 0:
    display(missing_df)

    # Stratégie : Imputer avec la médiane pour les variables numériques
    # (les NaN signifient 0 unités inscrites = étudiant probablement en difficulté)
    print("\n🔧 Stratégie d'imputation : Médiane pour les variables numériques")

    for col in missing_df.index:
        if col in numeric_features:
            median_val = X[col].median()
            X[col] = X[col].fillna(median_val)
            print(f"  {col}: NaN → {median_val:.3f} (médiane)")

    print("\n✅ Après imputation:")
    print(f"  Valeurs manquantes restantes: {X.isnull().sum().sum()}")
else:
    print("✅ Aucune valeur manquante!")

In [ ]:
# =============================================================================
# 6.3 Pipeline de Prétraitement et Split Train/Test
# =============================================================================

# Créer le preprocessor avec ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        (
            "cat",
            OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore"),
            categorical_features,
        ),
        ("bin", "passthrough", binary_features),
    ],
    remainder="drop",
)

# Split stratifié (préserve les proportions de classes)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("📊 Split Train/Test (stratifié):")
print("=" * 50)
print(f"\n  Train: {len(X_train)} samples ({len(X_train) / len(X) * 100:.0f}%)")
print(f"    - Dropout: {y_train.sum()} ({y_train.mean() * 100:.1f}%)")
print(f"    - Non-Dropout: {len(y_train) - y_train.sum()} ({(1 - y_train.mean()) * 100:.1f}%)")

print(f"\n  Test: {len(X_test)} samples ({len(X_test) / len(X) * 100:.0f}%)")
print(f"    - Dropout: {y_test.sum()} ({y_test.mean() * 100:.1f}%)")
print(f"    - Non-Dropout: {len(y_test) - y_test.sum()} ({(1 - y_test.mean()) * 100:.1f}%)")

# Vérifier que les proportions sont préservées
print(
    f"\n✅ Proportions préservées: Train {y_train.mean() * 100:.1f}% vs Test {y_test.mean() * 100:.1f}%"
)

In [ ]:
# =============================================================================
# 6.4 Modèle Baseline : Logistic Regression
# =============================================================================
# Pourquoi Logistic Regression comme baseline ?
# - Interprétable (coefficients = importance des features)
# - Rapide à entraîner
# - Référence pour comparer les modèles plus complexes

# Créer le pipeline complet (prétraitement + modèle)
baseline_model = Pipeline(
    [
        ("preprocessor", preprocessor),
        (
            "classifier",
            LogisticRegression(
                class_weight="balanced",  # Gère le déséquilibre des classes
                max_iter=1000,
                random_state=42,
            ),
        ),
    ]
)

# Entraîner le modèle
print("🔄 Entraînement du modèle baseline (Logistic Regression)...")
baseline_model.fit(X_train, y_train)
print("✅ Modèle entraîné!")

# Prédictions
y_pred = baseline_model.predict(X_test)
y_pred_proba = baseline_model.predict_proba(X_test)[:, 1]

# Métriques
print("\n" + "=" * 60)
print("📊 ÉVALUATION DU MODÈLE BASELINE")
print("=" * 60)

print("\n📋 Rapport de Classification:")
print("-" * 40)
print(classification_report(y_test, y_pred, target_names=["Non-Dropout", "Dropout"]))

# Métriques clés
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print("\n🎯 Métriques Prioritaires (classe Dropout):")
print("-" * 40)
print(f"  Recall (Dropout)  : {recall:.3f} ⭐ (priorité : ne pas manquer les à risque)")
print(f"  F1-Score (Dropout): {f1:.3f}")
print(f"  AUC-ROC           : {auc:.3f}")

In [ ]:
# =============================================================================
# 6.5 Visualisation des Résultats
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matrice de confusion
ax1 = axes[0]
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Non-Dropout", "Dropout"])
disp.plot(ax=ax1, cmap="Blues", values_format="d")
ax1.set_title(
    "Matrice de Confusion\n(Logistic Regression Baseline)",
    fontsize=14,
    fontweight="bold",
)

# Courbe ROC
ax2 = axes[1]
RocCurveDisplay.from_predictions(y_test, y_pred_proba, ax=ax2, color="#3498db", lw=2)
ax2.plot([0, 1], [0, 1], "k--", lw=1, label="Random (AUC=0.5)")
ax2.set_title(f"Courbe ROC (AUC = {auc:.3f})", fontsize=14, fontweight="bold")
ax2.legend(loc="lower right")

plt.tight_layout()
plt.show()

# Interprétation de la matrice de confusion
tn, fp, fn, tp = cm.ravel()
print("\n📊 Interprétation de la Matrice de Confusion:")
print("=" * 60)
print(f"  ✅ Vrais Négatifs (TN)  : {tn} Non-Dropouts correctement identifiés")
print(f"  ✅ Vrais Positifs (TP)  : {tp} Dropouts correctement identifiés")
print(f"  ⚠️ Faux Positifs (FP)   : {fp} Non-Dropouts classés Dropout (fausse alerte)")
print(f"  ❌ Faux Négatifs (FN)   : {fn} Dropouts manqués (risque principal!)")
print(
    f"\n💡 Sur {y_test.sum()} étudiants à risque, le modèle en identifie {tp} ({tp / y_test.sum() * 100:.1f}%)"
)

### Synthèse Phase 6 : Modèle Baseline

**Résultats du modèle Logistic Regression (Baseline)** :

| Métrique | Valeur | Interprétation |
|----------|--------|----------------|
| **Recall (Dropout)** | 83.1% | Sur 284 étudiants à risque, 236 sont identifiés |
| **F1-Score (Dropout)** | 81.4% | Bon équilibre précision/rappel |
| **AUC-ROC** | 93.1% | Excellente capacité de discrimination |
| **Accuracy** | 87.8% | Performance globale |

**Analyse des erreurs** :
- **48 Faux Négatifs** : Étudiants à risque non détectés (erreur critique ❌)
- **60 Faux Positifs** : Fausses alertes (coût acceptable)

**Conclusion** :
Le modèle baseline Logistic Regression atteint déjà des performances très satisfaisantes grâce :
1. Aux features engineered (`Success_Rate_Sem2` avec r = -0.705)
2. À la gestion du déséquilibre via `class_weight='balanced'`
3. À un bon prétraitement (StandardScaler + OneHotEncoder)

**Prochaines étapes suggérées** :
1. Tester d'autres modèles (Random Forest, XGBoost, etc.)
2. Optimiser les hyperparamètres avec GridSearchCV
3. Explorer SMOTE vs class_weight pour le déséquilibre
4. Ajuster le seuil de décision pour maximiser le Recall

## Phase 6.2 : Comparaison de Modèles et Optimisation

**Objectif** : Tester plusieurs algorithmes et optimiser le meilleur modèle.

### Modèles à comparer :
| Niveau | Modèle | Caractéristiques |
|--------|--------|------------------|
| Baseline | Logistic Regression | Déjà fait (Recall=83.1%, AUC=93.1%) |
| Intermédiaire | Random Forest | Robuste, feature importance native |
| Intermédiaire | Gradient Boosting | Performant, gère bien le déséquilibre |
| Intermédiaire | SVM (RBF) | Bon pour données non-linéaires |
| Avancé | XGBoost | État de l'art, très performant |

### Stratégie d'évaluation :
- **Cross-validation 5-fold stratifiée** pour comparer équitablement
- **Métrique principale** : F1-Score (équilibre précision/rappel)
- **Métrique secondaire** : Recall (ne pas manquer les dropouts)

### Comparaison de Moèles avec Cross-Validation

In [ ]:
# Calculer scale_pos_weight pour XGBoost (ratio classe majoritaire/minoritaire)
scale_pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()

# Définir les modèles à comparer
models = {
    "Logistic Regression": LogisticRegression(
        class_weight="balanced", max_iter=1000, random_state=42
    ),
    "Random Forest": RandomForestClassifier(
        n_estimators=100, class_weight="balanced", random_state=42, n_jobs=-1
    ),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "SVM (RBF)": SVC(kernel="rbf", class_weight="balanced", probability=True, random_state=42),
    "XGBoost": XGBClassifier(
        n_estimators=100,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        eval_metric="logloss",
        verbosity=0,
    ),
}

print(f"\nModèles à comparer : {len(models)}")
for name in models.keys():
    print(f"  - {name}")

In [ ]:
# =============================================================================
# 6.2.2 Cross-Validation 5-Fold Stratifiée
# =============================================================================
# Stratification : préserve les proportions de classes dans chaque fold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Stocker les résultats
results = {
    "Model": [],
    "F1 Mean": [],
    "F1 Std": [],
    "Recall Mean": [],
    "Recall Std": [],
    "AUC Mean": [],
    "AUC Std": [],
}

print("🔄 Cross-validation en cours...")
print("=" * 70)

for name, model in models.items():
    # Créer le pipeline pour chaque modèle
    pipeline = Pipeline([("preprocessor", preprocessor), ("classifier", model)])

    # Calculer les scores
    f1_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1)
    recall_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring="recall", n_jobs=-1)
    auc_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring="roc_auc", n_jobs=-1)

    # Stocker les résultats
    results["Model"].append(name)
    results["F1 Mean"].append(f1_scores.mean())
    results["F1 Std"].append(f1_scores.std())
    results["Recall Mean"].append(recall_scores.mean())
    results["Recall Std"].append(recall_scores.std())
    results["AUC Mean"].append(auc_scores.mean())
    results["AUC Std"].append(auc_scores.std())

    print(f"\n📊 {name}:")
    print(f"   F1-Score : {f1_scores.mean():.3f} (±{f1_scores.std():.3f})")
    print(f"   Recall   : {recall_scores.mean():.3f} (±{recall_scores.std():.3f})")
    print(f"   AUC-ROC  : {auc_scores.mean():.3f} (±{auc_scores.std():.3f})")

print("\n" + "=" * 70)
print("✅ Cross-validation terminée!")

In [ ]:
# =============================================================================
# 6.2.3 Tableau Comparatif des Modèles
# =============================================================================

results_df = pd.DataFrame(results)

# Trier par F1-Score
results_df = results_df.sort_values("F1 Mean", ascending=False)


# Formater pour l'affichage
def format_score(mean, std):
    return f"{mean:.3f} ±{std:.3f}"


display_df = pd.DataFrame(
    {
        "Modèle": results_df["Model"],
        "F1-Score": [
            format_score(m, s) for m, s in zip(results_df["F1 Mean"], results_df["F1 Std"])
        ],
        "Recall": [
            format_score(m, s) for m, s in zip(results_df["Recall Mean"], results_df["Recall Std"])
        ],
        "AUC-ROC": [
            format_score(m, s) for m, s in zip(results_df["AUC Mean"], results_df["AUC Std"])
        ],
    }
)

print("📊 Comparaison des Modèles (Cross-Validation 5-Fold)")
print("=" * 70)
display(display_df.reset_index(drop=True))

# Identifier le meilleur modèle
best_model_name = results_df.iloc[0]["Model"]
best_f1 = results_df.iloc[0]["F1 Mean"]
best_recall = results_df.iloc[0]["Recall Mean"]

print(f"\n🏆 Meilleur modèle (F1-Score) : {best_model_name}")
print(f"   F1 = {best_f1:.3f}, Recall = {best_recall:.3f}")

In [ ]:
# =============================================================================
# 6.2.4 Visualisation Comparative
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Préparer les données pour les graphiques
models_names = results_df["Model"].tolist()
colors = ["#3498db", "#2ecc71", "#e74c3c", "#9b59b6"]

# Graphique 1: F1-Score
ax1 = axes[0]
bars1 = ax1.barh(
    models_names,
    results_df["F1 Mean"],
    xerr=results_df["F1 Std"],
    color=colors,
    alpha=0.8,
    capsize=5,
)
ax1.set_xlabel("F1-Score", fontsize=12)
ax1.set_title("F1-Score par Modèle", fontsize=14, fontweight="bold")
ax1.set_xlim(0.65, 0.85)
for i, (v, s) in enumerate(zip(results_df["F1 Mean"], results_df["F1 Std"])):
    ax1.text(v + s + 0.005, i, f"{v:.3f}", va="center", fontsize=10)

# Graphique 2: Recall
ax2 = axes[1]
bars2 = ax2.barh(
    models_names,
    results_df["Recall Mean"],
    xerr=results_df["Recall Std"],
    color=colors,
    alpha=0.8,
    capsize=5,
)
ax2.set_xlabel("Recall", fontsize=12)
ax2.set_title("Recall (Dropout) par Modèle", fontsize=14, fontweight="bold")
ax2.set_xlim(0.65, 0.85)
for i, (v, s) in enumerate(zip(results_df["Recall Mean"], results_df["Recall Std"])):
    ax2.text(v + s + 0.005, i, f"{v:.3f}", va="center", fontsize=10)

# Graphique 3: AUC-ROC
ax3 = axes[2]
bars3 = ax3.barh(
    models_names,
    results_df["AUC Mean"],
    xerr=results_df["AUC Std"],
    color=colors,
    alpha=0.8,
    capsize=5,
)
ax3.set_xlabel("AUC-ROC", fontsize=12)
ax3.set_title("AUC-ROC par Modèle", fontsize=14, fontweight="bold")
ax3.set_xlim(0.85, 0.95)
for i, (v, s) in enumerate(zip(results_df["AUC Mean"], results_df["AUC Std"])):
    ax3.text(v + s + 0.002, i, f"{v:.3f}", va="center", fontsize=10)

plt.tight_layout()
plt.show()

print("\n💡 Observation : Les performances sont très proches entre les modèles.")
print(
    "   La Logistic Regression a le meilleur Recall (79.9%), importante pour détecter les dropouts."
)

### 6.2.5 Optimisation des Hyperparamètres

**Résultats de la comparaison** :
- Les 4 modèles ont des performances très similaires (F1 entre 0.77 et 0.79)
- **Logistic Regression** a le meilleur **Recall** (79.9%) - crucial pour notre objectif
- **SVM (RBF)** a le meilleur **F1-Score** (79.0%)

**Choix** : Nous allons optimiser **Logistic Regression** car :
1. Meilleur Recall (priorité pour détecter les dropouts)
2. Plus interprétable (coefficients analysables)
3. Plus rapide à entraîner et déployer

In [ ]:
# =============================================================================
# 6.2.6 GridSearchCV pour Logistic Regression
# =============================================================================

# Grille d'hyperparamètres pour Logistic Regression
param_grid = {
    "classifier__C": [0.01, 0.1, 1, 10, 100],  # Force de régularisation (inverse)
    "classifier__penalty": ["l1", "l2"],  # Type de régularisation
    "classifier__solver": ["saga"],  # Solver compatible avec l1 et l2
    "classifier__class_weight": ["balanced"],  # Gestion du déséquilibre
    "classifier__max_iter": [2000],  # Convergence
}

# Pipeline de base
base_pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(random_state=42)),
    ]
)

# GridSearchCV avec F1-Score comme métrique
print("🔄 Optimisation des hyperparamètres en cours...")
print("   (GridSearchCV avec 5-fold cross-validation)")
print("=" * 60)

grid_search = GridSearchCV(
    base_pipeline,
    param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="f1",
    n_jobs=-1,
    verbose=1,
)

grid_search.fit(X_train, y_train)

print("\n" + "=" * 60)
print("✅ Optimisation terminée!")
print("\n🏆 Meilleurs hyperparamètres:")
for param, value in grid_search.best_params_.items():
    print(f"   {param.replace('classifier__', '')}: {value}")
print(f"\n📊 Meilleur F1-Score (CV): {grid_search.best_score_:.3f}")

In [ ]:
# =============================================================================
# 6.2.7 Évaluation du Modèle Optimisé sur le Test Set
# =============================================================================

# Le meilleur modèle est déjà réentraîné sur tout le train set
best_model = grid_search.best_estimator_

# Prédictions sur le test set
y_pred_opt = best_model.predict(X_test)
y_pred_proba_opt = best_model.predict_proba(X_test)[:, 1]

# Métriques
print("=" * 60)
print("📊 ÉVALUATION DU MODÈLE OPTIMISÉ (Test Set)")
print("=" * 60)

print("\n📋 Rapport de Classification:")
print("-" * 40)
print(classification_report(y_test, y_pred_opt, target_names=["Non-Dropout", "Dropout"]))

# Métriques clés
recall_opt = recall_score(y_test, y_pred_opt)
f1_opt = f1_score(y_test, y_pred_opt)
auc_opt = roc_auc_score(y_test, y_pred_proba_opt)

print("\n🎯 Métriques Prioritaires (classe Dropout):")
print("-" * 40)
print(f"  Recall (Dropout)  : {recall_opt:.3f}")
print(f"  F1-Score (Dropout): {f1_opt:.3f}")
print(f"  AUC-ROC           : {auc_opt:.3f}")

# Comparaison avec le baseline
print("\n📈 Comparaison avec le Baseline:")
print("-" * 40)
print(
    f"  Recall    : {recall:.3f} → {recall_opt:.3f} ({'+' if recall_opt >= recall else ''}{(recall_opt - recall) * 100:.1f}%)"
)
print(
    f"  F1-Score  : {f1:.3f} → {f1_opt:.3f} ({'+' if f1_opt >= f1 else ''}{(f1_opt - f1) * 100:.1f}%)"
)
print(
    f"  AUC-ROC   : {auc:.3f} → {auc_opt:.3f} ({'+' if auc_opt >= auc else ''}{(auc_opt - auc) * 100:.1f}%)"
)

In [ ]:
# =============================================================================
# 6.2.8 Visualisation Finale : Matrice de Confusion et Feature Importance
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Matrice de confusion du modèle optimisé
ax1 = axes[0]
cm_opt = confusion_matrix(y_test, y_pred_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_opt, display_labels=["Non-Dropout", "Dropout"])
disp.plot(ax=ax1, cmap="Blues", values_format="d")
ax1.set_title(
    "Matrice de Confusion\n(Logistic Regression Optimisé)",
    fontsize=14,
    fontweight="bold",
)

# 2. Feature Importance (coefficients absolus)
ax2 = axes[1]

# Extraire les coefficients du modèle
classifier = best_model.named_steps["classifier"]
coefficients = classifier.coef_[0]

# Récupérer les noms des features après transformation
feature_names = (
    numeric_features
    + list(
        best_model.named_steps["preprocessor"]
        .named_transformers_["cat"]
        .get_feature_names_out(categorical_features)
    )
    + binary_features
)

# Créer un DataFrame pour visualisation
coef_df = pd.DataFrame(
    {
        "Feature": feature_names,
        "Coefficient": coefficients,
        "Abs_Coefficient": np.abs(coefficients),
    }
).sort_values("Abs_Coefficient", ascending=True)

# Top 10 features les plus importantes
top_features = coef_df.tail(10)
colors = ["#e74c3c" if c < 0 else "#2ecc71" for c in top_features["Coefficient"]]
ax2.barh(top_features["Feature"], top_features["Coefficient"], color=colors)
ax2.axvline(x=0, color="black", linestyle="-", linewidth=0.5)
ax2.set_xlabel("Coefficient (impact sur le dropout)", fontsize=11)
ax2.set_title(
    "Top 10 Features les Plus Influentes\n(Rouge = ↑ risque, Vert = ↓ risque)",
    fontsize=14,
    fontweight="bold",
)

plt.tight_layout()
plt.show()

# Interprétation
print("\n💡 Interprétation des Coefficients:")
print("=" * 60)
print("  🔴 Coefficient POSITIF → Augmente le risque de dropout")
print("  🟢 Coefficient NÉGATIF → Diminue le risque de dropout")

### Synthèse Phase 6.2 : Comparaison et Optimisation

**Résultats de la comparaison (Cross-Validation 5-Fold)** :

| Modèle | F1-Score | Recall | AUC-ROC |
|--------|----------|--------|---------|
| SVM (RBF) | 0.790 ±0.015 | 0.790 ±0.019 | 0.908 ±0.010 |
| Logistic Regression | 0.784 ±0.021 | 0.799 ±0.024 | 0.913 ±0.012 |
| Gradient Boosting | 0.783 ±0.013 | 0.726 ±0.012 | 0.911 ±0.013 |
| Random Forest | 0.774 ±0.018 | 0.713 ±0.012 | 0.906 ±0.013 |

**Choix et Optimisation** :
- Modèle choisi : **Logistic Regression** (meilleur Recall, interprétable)
- Hyperparamètres optimaux : `C=1`, `penalty='l1'`, `solver='saga'`

**Résultats du modèle optimisé (Test Set)** :

| Métrique | Baseline | Optimisé | Changement |
|----------|----------|----------|------------|
| Recall (Dropout) | 83.1% | 83.1% | +0.0% |
| F1-Score (Dropout) | 81.4% | 81.5% | +0.1% |
| AUC-ROC | 93.1% | 93.1% | +0.0% |

**Conclusion** :
1. Les 4 modèles testés ont des performances très similaires
2. L'optimisation des hyperparamètres n'a pas significativement amélioré les résultats
3. Cela confirme que les **features engineered** sont le facteur clé de la performance
4. Le modèle baseline était déjà proche de l'optimal grâce à l'EDA de qualité

**Top 5 Features les plus importantes** (Logistic Regression avec L1) :
1. `Success_Rate_Sem2` - Taux de réussite 2ème semestre
2. `Success_Rate_Sem1` - Taux de réussite 1er semestre
3. `Tuition fees up to date` - Frais de scolarité à jour
4. `Age_Group_36+` - Étudiants de 36 ans et plus
5. `Avg_Grade` - Moyenne des notes

## Phase 6.3 : Déploiement du Modèle

**Objectif** : Sauvegarder le modèle entraîné et créer une interface de prédiction réutilisable.

Cette phase comprend :
1. Sauvegarde du pipeline complet (prétraitement + modèle)
2. Fonction de prédiction pour de nouvelles données
3. Documentation et exemple d'utilisation

In [ ]:
# =============================================================================
# 6.3.1 SAUVEGARDE DU MODÈLE
# =============================================================================

# Créer le répertoire models/ s'il n'existe pas
os.makedirs("../models", exist_ok=True)

# Nom du fichier avec version
model_filename = "../models/dropout_predictor_v1.joblib"

# Sauvegarder le pipeline complet (prétraitement + modèle)
# best_model est créé dans la cellule d'évaluation (grid_search.best_estimator_)
joblib.dump(best_model, model_filename)

# Vérifier la taille du fichier
file_size = os.path.getsize(model_filename) / 1024  # en KB

print("=" * 60)
print("MODELE SAUVEGARDE AVEC SUCCES")
print("=" * 60)
print(f"\nFichier: {model_filename}")
print(f"Taille: {file_size:.1f} KB")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Informations sur le modèle sauvegardé
print("\nContenu du pipeline:")
print("   1. ColumnTransformer (prétraitement)")
print("      - StandardScaler pour variables numériques")
print("      - OneHotEncoder pour variables catégorielles")
print("      - Passthrough pour variables binaires")
print("   2. LogisticRegression (modèle optimisé)")
print(f"      - C={best_model.named_steps['classifier'].C}")
print(f"      - penalty={best_model.named_steps['classifier'].penalty}")
print(f"      - class_weight={best_model.named_steps['classifier'].class_weight}")

In [ ]:
# =============================================================================
# 6.3.2 FONCTION DE PRÉDICTION RÉUTILISABLE
# =============================================================================


def predict_dropout(student_data, model_path="../models/dropout_predictor_v1.joblib"):
    """
    Prédit le risque de dropout pour un étudiant.

    Parameters
    ----------
    student_data : dict ou pd.DataFrame
        Données de l'étudiant avec les features requises
    model_path : str
        Chemin vers le modèle sauvegardé

    Returns
    -------
    dict
        Prédiction (0/1), probabilité de dropout, et niveau de risque
    """
    import joblib
    import pandas as pd

    # Charger le modèle
    model = joblib.load(model_path)

    # Convertir en DataFrame si nécessaire
    if isinstance(student_data, dict):
        student_df = pd.DataFrame([student_data])
    else:
        student_df = student_data.copy()

    # Prédire
    prediction = model.predict(student_df)[0]
    proba = model.predict_proba(student_df)[0, 1]

    # Déterminer le niveau de risque
    if proba < 0.3:
        risk_level = "🟢 FAIBLE"
    elif proba < 0.5:
        risk_level = "🟡 MODÉRÉ"
    elif proba < 0.7:
        risk_level = "🟠 ÉLEVÉ"
    else:
        risk_level = "🔴 CRITIQUE"

    return {
        "prediction": "Dropout" if prediction == 1 else "Non-Dropout",
        "probability": round(proba, 3),
        "risk_level": risk_level,
    }


# Test avec un exemple du dataset
print("=" * 60)
print("🧪 TEST DE LA FONCTION DE PRÉDICTION")
print("=" * 60)

# Sélectionner un étudiant aléatoire du test set
test_sample = X_test.iloc[0:1]  # Premier étudiant du test set
actual_label = y_test.iloc[0]

print("\n📋 Données de l'étudiant test:")
print(test_sample.to_string())

result = predict_dropout(test_sample)

print("\n🎯 Résultat de la prédiction:")
print(f"   Prédiction     : {result['prediction']}")
print(f"   Probabilité    : {result['probability']:.1%}")
print(f"   Niveau de risque: {result['risk_level']}")
print(f"   Valeur réelle  : {'Dropout' if actual_label == 1 else 'Non-Dropout'}")
print(
    f"   Correct        : {'✅' if (result['prediction'] == 'Dropout') == (actual_label == 1) else '❌'}"
)

### Documentation - Utilisation du Modèle

#### Features Requises

Le modèle attend un DataFrame avec les 16 colonnes suivantes :

| Feature | Type | Description |
|---------|------|-------------|
| `Success_Rate_Sem1` | float | Ratio unités validées/inscrites au 1er semestre (0-1) |
| `Success_Rate_Sem2` | float | Ratio unités validées/inscrites au 2ème semestre (0-1) |
| `Avg_Grade` | float | Moyenne des notes sur les 2 semestres (0-20) |
| `Total_Approved` | int | Total des unités validées (sem1 + sem2) |
| `Age at enrollment` | int | Âge à l'inscription |
| `Admission grade` | float | Note d'admission (0-200) |
| `Performance_Trend` | float | Différence grade sem2 - grade sem1 |
| `Age_Group` | str | '17-20', '21-25', '26-35', ou '36+' |
| `Course_Domain` | str | 'Tech', 'Santé', 'Business', 'Social', 'Arts', 'Education' |
| `Marital_Binary` | str | 'Solo' ou 'Couple' |
| `Education_Level` | str | 'Secondaire', 'Supérieur', ou 'Autre' |
| `Tuition fees up to date` | int | 1 si frais à jour, 0 sinon |
| `Scholarship holder` | int | 1 si boursier, 0 sinon |
| `Debtor` | int | 1 si débiteur, 0 sinon |
| `Gender` | int | 1 = Homme, 0 = Femme |
| `Displaced` | int | 1 si déplacé, 0 sinon |

#### Exemple d'utilisation

```python
import joblib
import pandas as pd

# Charger le modèle
model = joblib.load('models/dropout_predictor_v1.joblib')

# Créer les données d'un étudiant
student = pd.DataFrame([{
    'Success_Rate_Sem1': 0.8,
    'Success_Rate_Sem2': 0.7,
    'Avg_Grade': 12.5,
    'Total_Approved': 10,
    'Age at enrollment': 20,
    'Admission grade': 140.0,
    'Performance_Trend': -1.0,
    'Age_Group': '17-20',
    'Course_Domain': 'Tech',
    'Marital_Binary': 'Solo',
    'Education_Level': 'Secondaire',
    'Tuition fees up to date': 1,
    'Scholarship holder': 0,
    'Debtor': 0,
    'Gender': 1,
    'Displaced': 0
}])

# Prédire
prediction = model.predict(student)[0]
probability = model.predict_proba(student)[0, 1]

print(f"Prédiction: {'Dropout' if prediction == 1 else 'Non-Dropout'}")
print(f"Probabilité de dropout: {probability:.1%}")
```

### Synthèse Phase 6.3 : Déploiement du Modèle

#### Réalisations

1. **Modèle Sauvegardé** : `models/dropout_predictor_v1.joblib` (5.7 KB)
   - Pipeline complet incluant prétraitement + modèle
   - Portable et réutilisable sans dépendance au notebook

2. **Fonction de Prédiction** : `predict_dropout()`
   - Accepte dict ou DataFrame
   - Retourne prédiction, probabilité et niveau de risque
   - Catégorisation du risque en 4 niveaux (Faible/Modéré/Élevé/Critique)

3. **Documentation** : Guide complet des features requises

#### Performance du Modèle Déployé

| Métrique | Valeur |
|----------|--------|
| **Recall (Dropout)** | 83.1% |
| **F1-Score** | 81.5% |
| **AUC-ROC** | 93.1% |
| **Précision globale** | 88% |

#### Top 3 Prédicteurs

1. `Success_Rate_Sem2` - Taux de réussite 2ème semestre
2. `Success_Rate_Sem1` - Taux de réussite 1er semestre  
3. `Tuition fees up to date` - Frais de scolarité à jour

---

## Conclusion de l'Analyse

Ce projet a démontré une approche complète de machine learning pour la prédiction du dropout étudiant :

1. **EDA approfondie** - Exploration des 37 variables, identification des patterns
2. **Feature Engineering** - Création de 9 nouvelles features pertinentes
3. **Modélisation** - Comparaison de 4 algorithmes, optimisation des hyperparamètres
4. **Déploiement** - Modèle portable et fonction de prédiction réutilisable

**Insight clé** : La qualité du feature engineering (ratios de réussite) a eu plus d'impact que le choix de l'algorithme. Les variables académiques du 2ème semestre sont les meilleurs prédicteurs du dropout.